In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from functools import partial
import nltk
from src.contextual_bart import ContextualisedBartModel,BartForContextualRecovery,SimplifiedBeamSearch
from src.dataset_processor import load_all_data
from src.utils import SmartCollator, get_args, setuptokenizer
from src.dataset_processor import (
    ContextGenerationDataset,
)
from transformers import BartTokenizer, BartConfig,BartForConditionalGeneration
from src.model_utils import CustomTrainer, get_training_arguments
import torch
from src.config import DATASET_PATH
from transformers.trainer_callback import EarlyStoppingCallback
import pickle as pk
import torch
from transformers import (    AutoTokenizer,
          AutoModelForSeq2SeqLM,
         LogitsProcessorList,    MinLengthLogitsProcessor, StoppingCriteriaList, MaxLengthCriteria,
         TopKLogitsWarper, TemperatureLogitsWarper,BeamSearchScorer,)

nltk.download("punkt")

DATASET_PATH = "summarisation_data/"

def generate_data():

    # load the dataset

    train_data_packet = load_all_data(DATASET_PATH, mode="train")
    dev_data_packet = load_all_data(DATASET_PATH, mode="dev")
    test_data_packet = load_all_data(DATASET_PATH,mode="test")

    print(f"Training Data size: {len(train_data_packet)}")
    print(f"Training Data size: {len(test_data_packet)}")
    return train_data_packet,dev_data_packet,test_data_packet

[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train_data_packet,dev_data_packet,test_data_packet = generate_data()

processing files:  ['summarisation_data/xsum_train.csv']
processing files:  ['summarisation_data/xsum_dev.csv']
processing files:  ['summarisation_data/xsum_test.csv']
Training Data size: 203083
Training Data size: 11322


In [4]:
import copy
from src.dataset_processor import ContextualGenerationData
import numpy as np

In [13]:
import copy
import random
def boundarySplit(data: ContextualGenerationData,
                  boundary_limits = (0.25,
                                     0.8),
                  max_context_tokens=[50,200],
                  nb_examples= 3
                  ):
    data_pack=[]
    max_context = np.linspace(max_context_tokens[0],max_context_tokens[-1],num=4)
    random.shuffle(max_context)
    for idx  in range(nb_examples):
        
        boundary_portion = np.round(np.random.uniform(
                    size=(1,),
                    low=boundary_limits[0],
                    high=boundary_limits[1],
                )[0],2)
        d= copy.deepcopy(data)
        d.boundary_proportion = (boundary_portion,int(max_context[idx]))
        data_pack.append(d)
    return data_pack

In [14]:
ff= boundarySplit(train_data_packet[0])

In [15]:
ff[0]

ContextualGenerationData(input='A Citroen C5, driven by a man, 80, from Stratford-upon-Avon, Warwickshire, and a Jaguar X-Type, driven by a man, 81, of Llandrindod Wells, collided near Walton at 17:30 GMT on Wednesday. Both men were taken to hospital, with the Citroen driver said to be critical. Police said a female passenger in the Jaguar died in hospital on Friday. She had been taken to Queen Elizabeth Hospital, in Birmingham - where the Citroen driver is being treated - after the accident, just over the border from the Herefordshire town of Kington. The force said her next of kin and the coroner have been informed.', output='A woman has died after a collision involving two cars on the A44 in Powys, Dyfed-Powys Police has said.', boundary=-1, focus_txt='', boundary_proportion=(0.79, 50))